# face_alignment

In [2]:
import os
os.getcwd()
#os.cd("./dataset/de

'C:\\Users\\CPB06GameN\\Python\\face_recognition'

In [13]:
import numpy as np
import dlib # face_landmark >> 얼굴 좌표 68개
import cv2

RIGHT_EYE = list(range(36, 42)) # 오른쪽 눈 좌표 범위
LEFT_EYE = list(range(42, 48)) # 왼쪽 눈 좌표 범위
EYES = list(range(36, 48)) # 눈 좌표 범위

predictor_file = 'model/shape_predictor_68_face_landmarks.dat'
image_file = 'dataset/JB/13.jpg'
MARGIN_RATIO = 1.5
OUTPUT_SIZE = (300, 300)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)

image = cv2.imread(image_file)
image_origin = image.copy() # 카피된 이미지(변수)로 작업

(image_height, image_width) = image.shape[:2] # 이미지 높이와 폭 저장
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # 노이즈 제거 > 흑백 사용

rects = detector(gray, 1) # 업스케일 레이어 수

def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

for (i, rect) in enumerate(rects):
    (x, y, w, h) = getFaceDimension(rect) # OpenCV에 맞게 좌표 수정 (Dlib >> OpenCV)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) # 얼굴에 사각형 그리기

    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()]) # 포인트 : 68개의 점(landmark) 가져옴
    show_parts = points[EYES] # 눈의 좌표만 보여줌

    right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int") # numpy로 좌표의 중간값(평균값)을 연산
    left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")
    print(right_eye_center, left_eye_center)
 
    cv2.circle(image, (right_eye_center[0,0], right_eye_center[0,1]), 5, (0, 0, 255), -1) # 크기가 5인 빨간점을 눈 중앙(numpy로 연산)에 찍음
    cv2.circle(image, (left_eye_center[0,0], left_eye_center[0,1]), 5, (0, 0, 255), -1)
    
    cv2.circle(image, (left_eye_center[0,0], right_eye_center[0,1]), 5, (0, 255, 0), -1) # 삼각함수로 기울기를 구하기 위해서 x값과 y값을 각각 가져와서 나머지 좌표에서 90도를 이루는 임의의 점을 찍음
    
    cv2.line(image, (right_eye_center[0,0], right_eye_center[0,1]),
             (left_eye_center[0,0], left_eye_center[0,1]), (0, 255, 0), 2) # 두 눈 사이의 
    cv2.line(image, (right_eye_center[0,0], right_eye_center[0,1]),
         (left_eye_center[0,0], right_eye_center[0,1]), (0, 255, 0), 1) # 밑변
    cv2.line(image, (left_eye_center[0,0], right_eye_center[0,1]), 
         (left_eye_center[0,0], left_eye_center[0,1]), (0, 255, 0), 1) # 높이

    eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0] # x변의 길이
    eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1] # 높이
    degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180 #아크-탄젠트(numpy)로 기울기 구하기 - 180 >> 각의 차이만큼 돌릴 예정이기 때문에 뻄
   # print("degree :", degree)
    
    eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2)) # 두 눈 사이의 거리 구하기 피타고라스의 정리 활용
    aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0] # 밑변
    scale = aligned_eye_distance / eye_distance # 눈 사이의 거리 >> 밑변의 거리가 되기 떄문에 scale을 구함
    #print("scale :", scale)
    
    eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0])//2,
                   (left_eye_center[0,1] + right_eye_center[0,1])//2) # 두 눈 사이의 좌표
    #print((left_eye_center[0,0] + right_eye_center[0,0]))
    #print((left_eye_center[0,1] + right_eye_center[0,1])//2)
    #eyes_center = map(int, eyes_center)
    print(eyes_center)
    
    cv2.circle(image, eyes_center, 5, (255, 0, 0), -1) # 두 눈 사이에 파란 점
            
    metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale) # 두 눈의 중간, degree, scale 를 metrix에 저장
    cv2.putText(image, "{:.5f}".format(degree), (right_eye_center[0,0], right_eye_center[0,1] + 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height), # 메트릭스값을 불러와서 이미지 회전
                            flags=cv2.INTER_CUBIC)
    
    #cv2.imshow("warpAffine", warped)
    (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)
    croped = warped[startY:endY, startX:endX]
    output = cv2.resize(croped, OUTPUT_SIZE) # 300,300 사이즈로 리사이즈
    cv2.imshow("output", output)

    for (i, point) in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

cv2.imshow("Face Alignment", image)
cv2.waitKey(0)
cv2.destroyAllWindows()


[[2133 1984]] [[2254 1975]]
(2193, 1979)


# use- alignment

In [2]:
import numpy as np
import dlib
import cv2
#from google.colab.patches import cv2_imshow #coloab
import datetime



RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

predictor_file = 'model/shape_predictor_68_face_landmarks.dat'
image_file = 'dataset/MZ/1.jpg'
MARGIN_RATIO = 1.5
OUTPUT_SIZE = (300, 300)


detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)

image = cv2.imread(image_file)
image_origin = image.copy()

(image_height, image_width) = image.shape[:2]
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

rects = detector(gray, 1)

def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

for (i, rect) in enumerate(rects):
    (x, y, w, h) = getFaceDimension(rect)
    cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[EYES]

    right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
    left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")
    print(right_eye_center, left_eye_center)
 
    cv2.circle(image, (right_eye_center[0,0], right_eye_center[0,1]), 5, (0, 0, 255), -1)
    cv2.circle(image, (left_eye_center[0,0], left_eye_center[0,1]), 5, (0, 0, 255), -1)
    
    cv2.circle(image, (left_eye_center[0,0], right_eye_center[0,1]), 5, (0, 255, 0), -1)
    
    cv2.line(image, (right_eye_center[0,0], right_eye_center[0,1]),
             (left_eye_center[0,0], left_eye_center[0,1]), (0, 255, 0), 2)
    cv2.line(image, (right_eye_center[0,0], right_eye_center[0,1]),
         (left_eye_center[0,0], right_eye_center[0,1]), (0, 255, 0), 1)
    cv2.line(image, (left_eye_center[0,0], right_eye_center[0,1]),
         (left_eye_center[0,0], left_eye_center[0,1]), (0, 255, 0), 1)

    eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
    eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
    degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180

    eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))
    aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]
    scale = aligned_eye_distance / eye_distance

    eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
            (left_eye_center[0,1] + right_eye_center[0,1]) // 2)
    cv2.circle(image, eyes_center, 5, (255, 0, 0), -1)
            
    metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale)
    cv2.putText(image, "{:.5f}".format(degree), (right_eye_center[0,0], right_eye_center[0,1] + 20),
     	 cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

    warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height),
        flags=cv2.INTER_CUBIC)
    
    cv2.imshow("warpAffine", warped) # 각도 회전
    #cv2_imshow(warped) #colab

    (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)
    croped = warped[startY:endY, startX:endX]
    output = cv2.resize(croped, OUTPUT_SIZE)
    cv2.imshow("output", output) # 최종
    #cv2_imshow(output) #colab
    
    # 현재시각을 불러와 문자열로저장
    now = datetime.datetime.now()
    nowDatetime = now.strftime('%Y-%m-%d %H:%M:%S')
    nowDatetime_path = now.strftime('%Y-%m-%d %H_%M_%S') # 파일이름으로는 :를 못쓰기 때문에 따로 만들어줌
    #cv2.imwrite("dataset/aligned/MZ/" + nowDatetime_path + ".jpg", output)


    for (i, point) in enumerate(show_parts):
        x = point[0,0]
        y = point[0,1]
        cv2.circle(image, (x, y), 1, (0, 255, 255), -1)

cv2.imshow("Face Alignment", image)
#cv2_imshow(image) #colab



cv2.waitKey(0)   
cv2.destroyAllWindows()
 

[[460 489]] [[705 385]]


# generate aligned_face photo

In [13]:
##### 파일 개수

import os
cwd = os.getcwd()
wd = cwd + "/dataset/JB"
print(wd)
jpg_list = [file for file in os.listdir(wd) if file.endswith(".jpg")]
print(jpg_list)
len(jpg_list)


C:\Users\CPB06GameN\Python\face_recognition/dataset/JB
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '3.jpg', '4.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']


26

In [6]:
import numpy as np
import dlib
import cv2

RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

dataset_path = '/dataset/JB'
output_path = '/dataset/aligned/JB'
number_images = 26
image_type = '.jpg'

predictor_file = './model/shape_predictor_68_face_landmarks.dat'
MARGIN_RATIO = 1.5
OUTPUT_SIZE = (300, 300)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)

def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())

def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

for idx in range(number_images):
    input_file = dataset_path + str(idx+1) + image_type

    # get RGB image from BGR, OpenCV format
    image = cv2.imread(input_file)
    if image is None:
        continue
    image_origin = image.copy()

    (image_height, image_width) = image.shape[:2]
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    rects = detector(gray, 1)

    for (i, rect) in enumerate(rects):
        (x, y, w, h) = getFaceDimension(rect)
        cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

    points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
    show_parts = points[EYES]

    right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
    left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

    eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
    eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
    degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180

    eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))
    aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]
    scale = aligned_eye_distance / eye_distance

    eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
        (left_eye_center[0,1] + right_eye_center[0,1]) // 2)
                    
    metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale)

    warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height),
        flags=cv2.INTER_CUBIC)

    (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)

    croped = warped[startY:endY, startX:endX]
    output = cv2.resize(croped, OUTPUT_SIZE)
    #output = warped[startY:endY, startX:endX]
            
    output_file = output_path + str(idx+1) + image_type
    cv2.imshow(output_file, output)
    cv2.imwrite(output_file, output)
        
        
cv2.waitKey(0)   
cv2.destroyAllWindows()


In [3]:
import cv2

image_file = 'dataset/MZ/1.jpg'

image = cv2.imread(image_file)
cv2.imshow("original", image)
#r_image = cv2.resize(image, dsize=(500, 500), interpolation = cv2.INTER_LINEAR)

#cv2.imshow("resize", r_image)

In [3]:
import cv2

input_file = "C:/Users/CPB06GameN/Python/face_recognition/dataset/JB/21.jpg"
image = cv2.imread(input_file)
cv2.imshow("",image)

# output = cv2.resize(croped, OUTPUT_SIZE)
#output = warped[startY:endY, startX:endX]

# output_file = output_path + str(idx+1) + image_type
# cv2.imshow(output_file)
            

# use it - face_aligned_image_generator

In [5]:
import numpy as np
import dlib
import cv2
import os

RIGHT_EYE = list(range(36, 42))
LEFT_EYE = list(range(42, 48))
EYES = list(range(36, 48))

dataset_paths = ['/dataset/JB/', '/dataset/MZ/', '/dataset/UZ/', '/dataset/kim min jae/','/dataset/son/','/dataset/tedy/']
output_paths = ['/dataset/aligned/JB/', '/dataset/aligned/MZ/', '/dataset/aligned/UZ/', '/dataset/aligned/kim min jae/', '/dataset/aligned/son/', '/dataset/aligned/tedy/']
image_type = '.jpg'

predictor_file = './model/shape_predictor_68_face_landmarks.dat'
MARGIN_RATIO = 1.5
OUTPUT_SIZE = (300, 300)

detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(predictor_file)


def getFaceDimension(rect):
    return (rect.left(), rect.top(), rect.right() - rect.left(), rect.bottom() - rect.top())


def getCropDimension(rect, center):
    width = (rect.right() - rect.left())
    half_width = width // 2
    (centerX, centerY) = center
    startX = centerX - half_width
    endX = centerX + half_width
    startY = rect.top()
    endY = rect.bottom() 
    return (startX, endX, startY, endY)    

for (i, dataset_path) in enumerate(dataset_paths):
    output_path = output_paths[i]
    
    ## 파일(이미지 개수)
    dataset_path = dataset_paths[i]
#     print(dataset_path)
#     wd = os.getcwd() + "/" + dataset_path
    dataset_path = os.getcwd() + dataset_path
    output_path = os.getcwd() + output_path
#     print(wd)
    
    jpg_list = [file for file in os.listdir(dataset_path) if file.endswith(image_type)]
    print(">>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>")
    print(jpg_list)
    number_images = len(jpg_list) # .jpg만
    ##
    
    for idx in range(number_images):
        input_file = dataset_path + str(idx+1) + image_type
#         input_file = dataset_path + str(idx+1) + image_type
        print("-------------------------------------------------------------------------------")
        print("input path: " + input_file)
        # get RGB image from BGR, OpenCV format
        image = cv2.imread(input_file)
        if image is None:
            print("load failed!!\n")
            continue
        image_origin = image.copy()

        (image_height, image_width) = image.shape[:2]
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        rects = detector(gray, 1)

        for (i, rect) in enumerate(rects):
            (x, y, w, h) = getFaceDimension(rect)
            if 'x' in locals():
                print("face detected!!")
            else :
                print("face not detected!!")
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)

            # face_landmark로 얼굴 탐지 및 랜드마크 좌표 획득
            points = np.matrix([[p.x, p.y] for p in predictor(gray, rect).parts()])
#             print(type(str(points.ndim)))
#             print("face_landmark: " + str(len(points)))            
            show_parts = points[EYES]

            right_eye_center = np.mean(points[RIGHT_EYE], axis = 0).astype("int")
            left_eye_center = np.mean(points[LEFT_EYE], axis = 0).astype("int")

            eye_delta_x = right_eye_center[0,0] - left_eye_center[0,0]
            eye_delta_y = right_eye_center[0,1] - left_eye_center[0,1]
            degree = np.degrees(np.arctan2(eye_delta_y,eye_delta_x)) - 180

            eye_distance = np.sqrt((eye_delta_x ** 2) + (eye_delta_y ** 2))
            aligned_eye_distance = left_eye_center[0,0] - right_eye_center[0,0]
            scale = aligned_eye_distance / eye_distance

            eyes_center = ((left_eye_center[0,0] + right_eye_center[0,0]) // 2,
                    (left_eye_center[0,1] + right_eye_center[0,1]) // 2)
                    
            metrix = cv2.getRotationMatrix2D(eyes_center, degree, scale)

            warped = cv2.warpAffine(image_origin, metrix, (image_width, image_height),
                flags=cv2.INTER_CUBIC)

            (startX, endX, startY, endY) = getCropDimension(rect, eyes_center)

            croped = warped[startY:endY, startX:endX]
            output = cv2.resize(croped, OUTPUT_SIZE)
            #output = warped[startY:endY, startX:endX]
            
            output_file = output_path + str(idx+1) + image_type
            print("output_path: " + output_file)
            print("saved!!\n")
            cv2.imshow(output_file, output)
            cv2.imwrite(output_file, output)
        
cv2.waitKey(0)   
cv2.destroyAllWindows()


>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
['1.jpg', '10.jpg', '11.jpg', '12.jpg', '13.jpg', '14.jpg', '15.jpg', '16.jpg', '17.jpg', '18.jpg', '19.jpg', '2.jpg', '20.jpg', '21.jpg', '22.jpg', '23.jpg', '24.jpg', '25.jpg', '26.jpg', '3.jpg', '5.jpg', '6.jpg', '7.jpg', '8.jpg', '9.jpg']
-------------------------------------------------------------------------------
input path: C:\Users\CPB06GameN\Python\face_recognition/dataset/JB/1.jpg
face detected!!
output_path: C:\Users\CPB06GameN\Python\face_recognition/dataset/aligned/JB/1.jpg
saved!!

-------------------------------------------------------------------------------
input path: C:\Users\CPB06GameN\Python\face_recognition/dataset/JB/2.jpg
face detected!!
output_path: C:\Users\CPB06GameN\Python\face_recognition/dataset/aligned/JB/2.jpg
saved!!

-------------------------------------------------------------------------------
input path: C:\Users\CPB06GameN\Python\face_recognition/dataset/JB/3.jpg
face de